In [1]:
# ==========================================
# 🧭 TASK 1: EDA + PREPROCESSING (DIRECT CSV)
# ==========================================
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

# 1️⃣ Load CSV file (no compression)
csv_path = '/content/drive/MyDrive/accepted_2007_to_2018Q4.csv'   # 👈 change path as needed
df = pd.read_csv(csv_path, low_memory=False)

print(f"✅ Loaded dataset with shape: {df.shape}")

# Optional: sample for faster testing
SAMPLE_FRAC = 2  # 5% sample for Colab testing (set to None for full)
RANDOM_STATE = 42
if 0 < SAMPLE_FRAC < 1:
    df = df.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"📉 Using sample of {len(df):,} rows")

# 2️⃣ Basic cleaning
if 'term' in df.columns:
    df['term'] = df['term'].astype(str).str.extract(r'(\d+)').astype(float)

if 'int_rate' in df.columns:
    df['int_rate'] = (
        df['int_rate'].astype(str).str.replace('%', '').replace('nan', '', regex=False)
    )
    df.loc[df['int_rate'] == '', 'int_rate'] = np.nan
    df['int_rate'] = pd.to_numeric(df['int_rate'], errors='coerce')

if 'emp_length' in df.columns:
    df['emp_length'] = (
        df['emp_length'].astype(str)
        .replace(['n/a', 'nan', 'None'], '0', regex=False)
        .str.replace(r'\+', '', regex=True)
        .str.replace(r'([<> ]+years?)', '', regex=True)
        .str.replace('year', '', regex=False)
        .str.strip()
    )
    df['emp_length'] = pd.to_numeric(df['emp_length'], errors='coerce').fillna(0)

if 'fico_range_low' in df.columns and 'fico_range_high' in df.columns:
    df['fico_mean'] = df[['fico_range_low', 'fico_range_high']].mean(axis=1)

if 'revol_util' in df.columns:
    df['revol_util'] = (
        df['revol_util'].astype(str).str.replace('%', '').replace('nan', '', regex=False)
    )
    df.loc[df['revol_util'] == '', 'revol_util'] = np.nan
    df['revol_util'] = pd.to_numeric(df['revol_util'], errors='coerce')

# 3️⃣ Define target
def map_status(s):
    s = str(s).lower()
    if 'fully paid' in s:
        return 0
    if 'charged off' in s or 'default' in s:
        return 1
    return np.nan

df['target'] = df['loan_status'].apply(map_status)
df = df.dropna(subset=['target']).reset_index(drop=True)
df['target'] = df['target'].astype(int)
print("🎯 Target variable distribution:")
print(df['target'].value_counts(normalize=True))

# 4️⃣ Select key features
features = [
    'loan_amnt', 'term', 'int_rate', 'installment',
    'grade', 'sub_grade', 'emp_length', 'home_ownership',
    'annual_inc', 'verification_status', 'purpose',
    'dti', 'delinq_2yrs', 'fico_mean', 'inq_last_6mths',
    'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc'
]
use_feats = [c for c in features if c in df.columns]
data = df[use_feats + ['target']].copy()

# 5️⃣ Identify numeric & categorical features
num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in use_feats if c not in num_cols]

numeric_features = [c for c in num_cols if c != 'target']
categorical_features = cat_cols

# 6️⃣ Build preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 7️⃣ Fit & transform
X = preprocessor.fit_transform(data.drop(columns=['target']))
y = data['target'].values

# 8️⃣ Recreate column names
ohe = preprocessor.named_transformers_['cat'].named_steps['ohe']
cat_feature_names = ohe.get_feature_names_out(categorical_features)
num_feature_names = numeric_features
processed_columns = np.concatenate([num_feature_names, cat_feature_names])

processed_df = pd.DataFrame(X, columns=processed_columns)
processed_df['target'] = y


✅ Loaded dataset with shape: (2260701, 151)
🎯 Target variable distribution:
target
0    0.800193
1    0.199807
Name: proportion, dtype: float64


In [2]:

# 9️⃣ Save clean dataset
output_path = "/content/drive/MyDrive/loan_data_processed.csv"
processed_df.to_csv(output_path, index=False)
print(f"💾 Saved processed dataset to {output_path}")
print("✅ Final processed shape:", processed_df.shape)

# 🔟 (Optional) Save preprocessor for reuse in modeling
joblib.dump(preprocessor, "/content/drive/MyDrive/loan_preprocessor.pkl")
print("🧠 Preprocessor saved for later modeling.")

💾 Saved processed dataset to /content/drive/MyDrive/loan_data_processed.csv
✅ Final processed shape: (1348099, 81)
🧠 Preprocessor saved for later modeling.


In [3]:
import pandas as pd

path = "/content/drive/MyDrive/loan_data_processed.csv"
df = pd.read_csv(path, nrows=1348099)  # sample to inspect
print("✅ Preview shape:", df.shape)

print("\n🎯 Target distribution:")
print(df['target'].value_counts(normalize=True).rename('proportion'))

print("\n🚫 Missing values (top 5):")
print(df.isna().mean().sort_values(ascending=False).head())

print("\n📊 Basic stats:")
print(df.describe().T.head(10))


✅ Preview shape: (1348099, 81)

🎯 Target distribution:
target
0    0.800193
1    0.199807
Name: proportion, dtype: float64

🚫 Missing values (top 5):
loan_amnt      0.0
term           0.0
int_rate       0.0
installment    0.0
emp_length     0.0
dtype: float64

📊 Basic stats:
                    count          mean  std       min       25%       50%  \
loan_amnt       1348099.0  3.923510e-17  1.0 -1.595776 -0.738171 -0.276384   
term            1348099.0  2.201467e-16  1.0 -0.563493 -0.563493 -0.563493   
int_rate        1348099.0 -4.467025e-16  1.0 -1.664307 -0.732647 -0.105245   
installment     1348099.0  4.404725e-16  1.0 -1.655268 -0.724665 -0.239918   
emp_length      1348099.0  4.790672e-17  1.0 -1.460159 -0.940035  0.100212   
annual_inc      1348099.0  4.002149e-16  1.0 -1.090315 -0.436021 -0.160716   
dti             1348099.0 -7.585172e-16  1.0 -1.728004 -0.581325 -0.059537   
delinq_2yrs     1348099.0 -2.023949e-18  1.0 -0.361869 -0.361869 -0.361869   
fico_mean       134809